In [1]:
# -*- coding: utf-8
# Abraji (https://www.abraji.org.br)
# Reinaldo Chaves (reinaldo@abraji.org.br)
# Programa para buscas de processos no STF a partir de nomes
#

In [4]:
from bs4 import BeautifulSoup as bs
import requests
import pandas as pd
import math
import os

In [5]:
print(os.getcwd())

/home/reinaldo/Documentos/Code/abraji/coda2019/repo/raspatribunais_stf


In [6]:
# Amostra de candidatos

In [7]:
candidatos = pd.read_csv('exemplo_candidatos_presidente.csv', encoding='utf-8', converters={'CPF_CANDIDATO': lambda x: str(x)})

In [8]:
candidatos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4 entries, 0 to 3
Data columns (total 20 columns):
ANO_ELEICAO                    4 non-null int64
SIGLA_UF                       4 non-null object
SIGLA_UE                       4 non-null object
DESCRICAO_UE                   4 non-null object
CODIGO_CARGO                   4 non-null int64
DESCRICAO_CARGO                4 non-null object
DESCRICAO_SEXO                 4 non-null object
NOME_CANDIDATO                 4 non-null object
NOME_URNA_CANDIDATO            4 non-null object
SIGLA_PARTIDO                  4 non-null object
SEQUENCIAL_CANDIDATO           4 non-null int64
NUMERO_CANDIDATO               4 non-null int64
CPF_CANDIDATO                  4 non-null object
CODIGO_MUNICIPIO_NASCIMENTO    4 non-null int64
NOME_MUNICIPIO_NASCIMENTO      4 non-null object
DATA_NASCIMENTO                4 non-null object
SIGLA_UF_NASCIMENTO            4 non-null object
COD_SIT_TOT_TURNO              4 non-null int64
DESC_SIT_TOT_TURNO     

In [9]:
candidatos.head()

,ANO_ELEICAO,SIGLA_UF,SIGLA_UE,DESCRICAO_UE,CODIGO_CARGO,DESCRICAO_CARGO,DESCRICAO_SEXO,NOME_CANDIDATO,NOME_URNA_CANDIDATO,SIGLA_PARTIDO,SEQUENCIAL_CANDIDATO,NUMERO_CANDIDATO,CPF_CANDIDATO,CODIGO_MUNICIPIO_NASCIMENTO,NOME_MUNICIPIO_NASCIMENTO,DATA_NASCIMENTO,SIGLA_UF_NASCIMENTO,COD_SIT_TOT_TURNO,DESC_SIT_TOT_TURNO,nome
0,2018,BR,BR,BRASIL,1,PRESIDENTE,MASCULINO,JOSE MARIA EYMAEL,EYMAEL,DC,280000624085,27,01061712834,-3,PORTO ALEGRE,02/11/1939,RS,4,NÃO ELEITO,JOSE MARIA EYMAEL
1,2014,BR,BR,BRASIL,1,PRESIDENTE,MASCULINO,AÉCIO NEVES DA CUNHA,AÉCIO NEVES,PSDB,280000000085,45,66728983791,-3,BELO HORIZONTE,10/03/1960,MG,4,NÃO ELEITO,AECIO NEVES DA CUNHA
2,2018,BR,BR,BRASIL,1,PRESIDENTE,MASCULINO,LUIZ INÁCIO LULA DA SILVA,LULA,PT,280000625869,13,07068093868,-3,GARANHUNS,06/10/1945,PE,-1,#NULO#,LUIZ INACIO LULA DA SILVA
3,2018,BR,BR,BRASIL,1,PRESIDENTE,MASCULINO,JAIR MESSIAS BOLSONARO,JAIR BOLSONARO,PSL,280000614517,17,45317828791,-3,CAMPINAS,21/03/1955,SP,1,ELEITO,JAIR MESSIAS BOLSONARO


In [8]:
# Função para pegar a paginação de processos

In [10]:
def conta_pagina(politico_conta, nome):
    
    # Faz os testes para o request da página
    try:
        r = requests.get(politico_conta)
    except (requests.exceptions.HTTPError, requests.exceptions.RequestException, requests.exceptions.ConnectionError, requests.exceptions.Timeout) as e:
        print(str(e))
        return 
    except Exception as e:
        print("Exceção")
        return
    
    # Captura o número de processos
    soup = bs(r.content, 'lxml')
    qt = soup.select_one('#totalProc')['value']
    qt = int(qt)
    
    # Testa se existe mais de 1 página de processos
    if qt / 20 <= 1:
        print(nome + " tem 1 página")
        return ("0", "1")
    else:
        conta = qt / 20
        # Função para retornar o menor valor integral 
        x = math.ceil(conta)
        print(nome + " tem " + str(x) + " páginas")
        return (qt, x)

In [11]:
# Função para extrair número CNJ e dados gerais

In [12]:
def le_lista(politico_link, nome, cpf):
    print(nome)
    
    # Faz os testes para o request da página
    try:
        res = requests.get(politico_link)
    except (requests.exceptions.HTTPError, requests.exceptions.RequestException, requests.exceptions.ConnectionError, requests.exceptions.Timeout) as e:
        print(str(e))
        return 
    except Exception as e:
        print("Exceção")
        return

    # Este site é preciso decodificar o resultado
    html = res.content.decode('utf-8') 
    
    # Faz o parseamento
    soup =  bs(html, "lxml")

    # Captura a tabela inteira
    parts = soup.find_all('table', {'class': 'table m-b-0'})
    
    #pag = soup.find_all('div', {'id': 'total'})
    #print(pag)
        
    # Itera na tabela toda - no caso há só uma
    for part in parts:
        # Pega o conteúdo das linhas, que tem cada informação com a tag "td", de toda coluna
        lines = part.find_all('td')
        
        
        # Divide cada linha em partes - as colunas
        # O range estabele uma posição de começo
        # Uma posição final, que sempre será um número a menos
        # E pode ter um step, ou seja, o incremento - aqui de sete em sete
        chunks = [lines[x:x+7] for x in range(0, len(lines), 7)]
        
        # Acessa cada parte que foi dividida
        for vez in chunks:
            
            # Atribui cada parte a um valor no dicionário
            processos.append({'nome_completo': nome, 
                                'cpf': cpf, 
                                'identificacao': vez[0].string, 
                                'identificacao_link': "http://portal.stf.jus.br/processos/" + vez[0].a.get('href'), 
                                'nome_no_tribunal': vez[1].string, 
                                'numero_unico': vez[2].string, 
                                'data_atuacao': vez[3].string, 
                                'meio': vez[4].string,
                                'publicidade': vez[5].string,
                                'tramite': vez[6].string}
                            )    
 
    return processos

In [13]:
# Link geral para buscas
link_stf = 'http://portal.stf.jus.br/processos/listarPartes.asp?termo='

In [14]:
# Link de outra chamada XHR que você pode encontrar na aba Network em Inspecionar (botão direito do mouse ou Ctrl+Shift+I)
# XMLHttpRequest (XHR) é uma API disponível para linguagens de script de navegador da Web, como JavaScript. 
# É usado para enviar solicitações HTTP ou HTTPS para um servidor da Web e carregar os dados de resposta do servidor novamente no script.
link_stf_xhr = 'https://portal.stf.jus.br/processos/totalProcessosPartes.asp?termo='

In [15]:
# Lista vazia para acumular dados

In [16]:
processos = []

In [17]:
# Faz iteração com os candidatos

In [18]:
for num, row in candidatos.iterrows():
    nome = row['nome']
    
    cpf = row['CPF_CANDIDATO']
    politico_link = link_stf + nome + '&total=0'
    politico_conta = link_stf_xhr + nome + '&total=0'
    
    # Chama função para pegar o número de páginas de cada político
    total_proc, num_pagina = conta_pagina(politico_conta, nome)
    
    # Mostra a quantidade
    print(num_pagina)
    
    # Se a pessoa tem apenas uma página
    if num_pagina == '1':
        le_lista(politico_link, nome, cpf)
    
    # Se possui mais de uma faz uma iteração
    else:
        for item in range(1, num_pagina+1):
            print(item)
            vez = str(item)
            total_proc = str(total_proc)
            # Altera o final da cada URL
            politico_link = link_stf + nome + "&total=" + total_proc + "&paginaAtual=" + vez
            print(politico_link)
            le_lista(politico_link, nome, cpf)

JOSE MARIA EYMAEL tem 1 página
1
JOSE MARIA EYMAEL
AECIO NEVES DA CUNHA tem 2 páginas
2
1
http://portal.stf.jus.br/processos/listarPartes.asp?termo=AECIO NEVES DA CUNHA&total=35&paginaAtual=1
AECIO NEVES DA CUNHA
2
http://portal.stf.jus.br/processos/listarPartes.asp?termo=AECIO NEVES DA CUNHA&total=35&paginaAtual=2
AECIO NEVES DA CUNHA
LUIZ INACIO LULA DA SILVA tem 13 páginas
13
1
http://portal.stf.jus.br/processos/listarPartes.asp?termo=LUIZ INACIO LULA DA SILVA&total=255&paginaAtual=1
LUIZ INACIO LULA DA SILVA
2
http://portal.stf.jus.br/processos/listarPartes.asp?termo=LUIZ INACIO LULA DA SILVA&total=255&paginaAtual=2
LUIZ INACIO LULA DA SILVA
3
http://portal.stf.jus.br/processos/listarPartes.asp?termo=LUIZ INACIO LULA DA SILVA&total=255&paginaAtual=3
LUIZ INACIO LULA DA SILVA
4
http://portal.stf.jus.br/processos/listarPartes.asp?termo=LUIZ INACIO LULA DA SILVA&total=255&paginaAtual=4
LUIZ INACIO LULA DA SILVA
5
http://portal.stf.jus.br/processos/listarPartes.asp?termo=LUIZ INACIO LU

In [19]:
df_processos = pd.DataFrame(processos)

In [20]:
df_processos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 335 entries, 0 to 334
Data columns (total 10 columns):
nome_completo         335 non-null object
cpf                   335 non-null object
identificacao         335 non-null object
identificacao_link    335 non-null object
nome_no_tribunal      335 non-null object
numero_unico          335 non-null object
data_atuacao          335 non-null object
meio                  335 non-null object
publicidade           335 non-null object
tramite               335 non-null object
dtypes: object(10)
memory usage: 26.3+ KB


In [21]:
df_processos.reset_index().head()

,index,nome_completo,cpf,identificacao,identificacao_link,nome_no_tribunal,numero_unico,data_atuacao,meio,publicidade,tramite
0,0,JOSE MARIA EYMAEL,01061712834,ARE 914578,http://portal.stf.jus.br/processos/detalhe.asp...,JOSÉ MARIA EYMAEL,Sem número único,10/09/2015,Físico,Público,Não
1,1,JOSE MARIA EYMAEL,01061712834,AI 730702,http://portal.stf.jus.br/processos/detalhe.asp...,JOSÉ MARIA EYMAEL,Sem número único,30/10/2008,Físico,Público,Não
2,2,JOSE MARIA EYMAEL,01061712834,MS 21736,http://portal.stf.jus.br/processos/detalhe.asp...,JOSE MARIA EYMAEL,0001529-69.1993.0.01.0000,25/08/1993,Físico,Público,Não
3,3,JOSE MARIA EYMAEL,01061712834,MS 21648,http://portal.stf.jus.br/processos/detalhe.asp...,JOSE MARIA EYMAEL,0000193-30.1993.0.01.0000,09/02/1993,Físico,Público,Não
4,4,JOSE MARIA EYMAEL,01061712834,RE 119465,http://portal.stf.jus.br/processos/detalhe.asp...,JOSE MARIA EYMAEL,Sem número único,22/03/1989,Físico,Público,Não


In [22]:
# Mantem o número CNJ mais recente e todos os "Sem número único"

In [ ]:
# Retira espaços em branco

In [23]:
df_processos['numero_unico'] = df_processos['numero_unico'].str.strip()

In [ ]:
# Separa em dois dataframes

In [24]:
tmp_df1 = df_processos[df_processos['numero_unico'] == "Sem número único"]

In [25]:
tmp_df2 = df_processos[df_processos['numero_unico']!='Sem número único']

In [ ]:
# Mantém a última citação de um número único

In [26]:
tmp_df2 = tmp_df2.drop_duplicates(['numero_unico'], keep='last')

In [ ]:
# Junta os dois dataframes

In [27]:
new_df = pd.concat([tmp_df1, tmp_df2])

In [28]:
new_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 328 entries, 0 to 334
Data columns (total 10 columns):
nome_completo         328 non-null object
cpf                   328 non-null object
identificacao         328 non-null object
identificacao_link    328 non-null object
nome_no_tribunal      328 non-null object
numero_unico          328 non-null object
data_atuacao          328 non-null object
meio                  328 non-null object
publicidade           328 non-null object
tramite               328 non-null object
dtypes: object(10)
memory usage: 28.2+ KB


In [29]:
new_df.to_csv('processos_politicos_stf.csv', index=False)